In [1]:
'''
from docx import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from chromadb import Client
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from dotenv import load_dotenv, find_dotenv
import openai
from sentence_transformers import CrossEncoder
import numpy as np
from openai import OpenAI
'''

ModuleNotFoundError: No module named 'docx'

In [4]:
from pypdf import PdfReader

In [7]:
reader = PdfReader("../data/2017-ith-chapter7.pdf")
pdf_texts = [p.extract_text().strip() for p in reader.pages]

pdf_texts = [text for text in pdf_texts if text]

print(pdf_texts[0])

INTERNATIONAL TRAVEL AND HEALTH - CHAPTER 7 
Malaria  
 
 
7.1  Background 
Malaria is a common and life-threatening disease in  many tropical and subtropical areas. There 
is currently a risk of malaria transmission in 91 c ountries and  territories, and these are visited 
by more than 125 million international travellers e very  year. 
Each year many international travellers fall ill wi th malaria while visiting countries/territories 
where malaria is endemic, and well  over 10 000 are reported to become ill with malaria  after 
returning home. However, under-reporting means that  the real figure may  be considerably 
higher. International travellers to countries/terri tories with ongoing local malaria transmission 
arriving from  countries with no transmission are at high risk of malaria infection and its 
consequences because they lack immunity. Migrants  from countries/territories with malaria 
transmission living in malaria-free countries and r eturning to their home countries to

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter

In [19]:
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n",". "," ",""],
    chunk_size = 1000,
    chunk_overlap=0
)

character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

print(character_split_texts[10])
print(f"\nTotal chunks: {len(character_split_texts)}")

infection and believe in the  benefit of prevention strategies. Late-onset vivax and ovale 
malaria may occur despite effective prophylaxis bec ause these parasites  cause relapses that 
cannot be prevented with medicines that are current ly recommended for chemoprophylaxis. 
Malaria risk is not evenly distributed where the di sease is prevalent. Travellers to any 
country/territory in which malaria  transmission varies by area should seek advice on t he risk 
of infection in the particular zones that they will  be visiting. If specific  information is not 
available before travelling, it is recommended that  precautions appropriate for the highest 
reported risk in the  country/territory should be taken. This  applies particularly to individuals 
backpacking to remote places and visiting areas whe re health facilities are not readily 
available.  Travellers staying in rural areas at night may be a t highest risk. 
 
 
7.2 Precautions

Total chunks: 70


In [21]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

print(token_split_texts[10])
print(f"\nTotal chunks: {len(token_split_texts)}")

/home/mahbubah/Desktop/projets/precision-RAG/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


infection and believe in the benefit of prevention strategies. late - onset vivax and ovale malaria may occur despite effective prophylaxis bec ause these parasites cause relapses that cannot be prevented with medicines that are current ly recommended for chemoprophylaxis. malaria risk is not evenly distributed where the di sease is prevalent. travellers to any country / territory in which malaria transmission varies by area should seek advice on t he risk of infection in the particular zones that they will be visiting. if specific information is not available before travelling, it is recommended that precautions appropriate for the highest reported risk in the country / territory should be taken. this applies particularly to individuals backpacking to remote places and visiting areas whe re health facilities are not readily available. travellers staying in rural areas at night may be a t highest risk. 7. 2 precautions

Total chunks: 73


In [25]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()
print(embedding_function([token_split_texts[10]]))

[[0.07742436230182648, 0.04551458731293678, 0.038965608924627304, -0.05207369104027748, 0.10041967034339905, 0.04394061863422394, 0.06373430043458939, 0.12069519609212875, -0.03627041354775429, 0.07096431404352188, 0.03404441103339195, -0.0037035441491752863, -0.006301800720393658, 0.05267469957470894, 0.011027327738702297, -0.018824612721800804, 0.027707455679774284, -0.042254459112882614, -0.05846555158495903, -0.0339440181851387, -0.02181548997759819, 0.00870450958609581, 0.017603926360607147, 0.047661568969488144, -0.09634354710578918, -0.002335143508389592, 0.005721183959394693, 0.014127206988632679, 0.08240914344787598, 0.03719819709658623, -0.02470913901925087, 0.07127659022808075, -0.11397988349199295, -0.06065041571855545, -0.09781268984079361, 0.056620821356773376, 0.005463159643113613, -0.08404593169689178, 0.01144388783723116, 0.035839926451444626, 0.030277980491518974, -0.023018136620521545, -0.028705265372991562, -0.0764763280749321, -0.040207866579294205, -0.105443589389

In [27]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("disease", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()

73

In [28]:
query = "How do we prevent malaria?"

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print(document)
    print('\n')

travellers and their advisers should note the five principles – the abcde – of malaria protection : • be aware of the risk, the incubation period, the possib ility of delayed onset, and the main symptoms. • avoid being bitten by mosquitoes, especially between dusk and da wn. • take antimalarial drugs ( chemoprophylaxis ) when appropriate, at regular inter vals to prevent acute malaria attacks. • immediately seek diagnosis and treatment if a fever develops 1 week o r more after entering an area where there is a malaria risk and up to 3 months ( or, rarely, later ) after departure from a risk area. • avoid outdoor activities in environments that are mosquito breeding places, such as swamps or marshy areas, especially in late evenings and at night. 7. 2. 1 protection against mosquito bites all travellers should be advised that personal prot ection from mosquito bites between dusk and dawn is their first line of defence against malaria.


infection and believe in the benefit of prevention 